In [1]:
import pygame, sys, random
import numpy as np
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import Adam
import math
import matplotlib.pyplot as plt
import pylab

num_episodes = 5000

agentRadius = 10

# get size of state and action from environment

boundaryPos = [400, 590]
boundaryLength = [100,100]
initPosAgent = [boundaryPos[0] + boundaryLength[0] / 2, boundaryPos[1] + boundaryLength[1] / 2]
goalPos = [boundaryPos[0] + boundaryLength[0], boundaryPos[1]]

moveObstacles = True
action_size = 8
obsNumber = 1
state_size = obsNumber * 2 + 1
# state_size = obsNumber + 1

Using TensorFlow backend.


In [2]:
# A2C(Advantage Actor-Critic) agent
class A2CAgent:
    def __init__(self, state_size, action_size):
        self.load_model = True
        # get size of state and action
        self.state_size = state_size
        self.action_size = action_size
        self.value_size = 1

        # These are hyper parameters for the Policy Gradient
        self.discount_factor = 0.99
        self.actor_lr = 0.00002
        self.critic_lr = 0.00005

        # create model for policy network
        self.actor = self.build_actor()
        self.critic = self.build_critic()

        if self.load_model:
            self.actor.load_weights("./Practice004_DataSave/Actor.h5")
            self.critic.load_weights("./Practice004_DataSave/Critic.h5")

    # approximate policy and value using Neural Network
    # actor: state is input and probability of each action is output of model
    def build_actor(self):
        actor = Sequential()
        actor.add(Dense(128, input_dim=self.state_size, activation='relu', kernel_initializer='glorot_normal'))
        actor.add(Dense(self.action_size, activation='softmax', kernel_initializer='glorot_normal'))
        actor.summary()
        # See note regarding crossentropy in cartpole_reinforce.py
        actor.compile(loss='categorical_crossentropy', optimizer=Adam(lr=self.actor_lr))
        return actor

    # critic: state is input and value of state is output of model
    def build_critic(self):
        critic = Sequential()
        critic.add(Dense(128, input_dim=self.state_size, activation='relu', kernel_initializer='glorot_normal'))
        critic.add(Dense(self.value_size, activation='linear', kernel_initializer='glorot_normal'))
        critic.summary()
        critic.compile(loss="mse", optimizer=Adam(lr=self.critic_lr))
        return critic

    # using the output of policy network, pick action stochastically
    def get_action(self, state):
        policy = self.actor.predict(state, batch_size=1).flatten()
#         print policy
        return np.random.choice(self.action_size, 1, p=policy)[0]

    # update policy network every episode
    def train_model(self, state, action, reward, next_state, done):
        target = np.zeros((1, self.value_size))
        advantages = np.zeros((1, self.action_size))

        value = self.critic.predict(state)[0]
        next_value = self.critic.predict(next_state)[0]

        if done:
            advantages[0][action] = reward - value
            target[0][0] = reward
        else:
            advantages[0][action] = reward + self.discount_factor * (next_value) - value
            target[0][0] = reward + self.discount_factor * next_value

        self.actor.fit(state, advantages, epochs=1, verbose=0)
        self.critic.fit(state, target, epochs=1, verbose=0)

In [3]:
def ckWall(xPos, yPos):
    flagWall = 0
    if(xPos < boundaryPos[0]):
        xPos = boundaryPos[0]
        flagWall = -1
    elif(xPos > boundaryPos[0] + boundaryLength[0]):
        xPos = boundaryPos[0] + boundaryLength[0]
        flagWall = -1
    if(yPos < boundaryPos[1]):
        yPos = boundaryPos[1]
        flagWall = -1
    elif(yPos > boundaryPos[1] + boundaryLength[1]):
        yPos = boundaryPos[1] + boundaryLength[1]
        flagWall = -1
        
    return [xPos, yPos, flagWall]

In [4]:
def ckInit(initLoc, obsLoc):
    if math.sqrt((initLoc[0] - obsLoc[0])**2 + (initLoc[1] - obsLoc[1])**2) <= 2*agentRadius + 1:
        if initLoc[0] - obsLoc[0] < 0:
            obsLoc[0] += 1
        else:
            obsLoc[0] -= 1
        if initLoc[1] - obsLoc[1] < 0:
            obsLoc[1] += 1
        else:
            obsLoc[1] -= 1
    if math.sqrt((goalPos[0] - obsLoc[0])**2 + (goalPos[1] - obsLoc[1])**2) <= 2*agentRadius + 1:
        obsLoc[0] -= 1
        obsLoc[1] += 1
    return obsLoc

In [5]:
def stateGenerator(obsPosition, agtPosition):
    returnSum = []
    for i in range(0,obsNumber):
#         returnSum = returnSum + [math.sqrt((agtPosition[0] - obsPosition[i][0])**2 + (agtPosition[1] - obsPosition[i][1])**2)]
        returnSum = returnSum + [agtPosition[0] - obsPosition[i][0], agtPosition[1] - obsPosition[i][1]]
#     returnSum = returnSum + [agtPosition[0] - 640, agtPosition[1] - 450]
    returnSum = returnSum + [math.sqrt((agtPosition[0] - goalPos[0])**2 + (agtPosition[1] - goalPos[1])**2)]
    returnSum = np.reshape(returnSum, [1, state_size])
    return returnSum

In [6]:
def takeAction(action):
    xAction = 0
    yAction = 0
    if action == 0:
        xAction = 1
    elif action == 1:
        xAction = 1
        yAction = 1
    elif action == 2:
        xAction = 1
        yAction = -1            
    elif action == 3:
        xAction = -1
        yAction = 1
    elif action == 4:
        xAction = -1
    elif action == 5:
        xAction = -1
        yAction = -1
    elif action == 6:
        yAction = -1
    elif action == 7:
        yAction = 1
        
    return [xAction, yAction]

In [ ]:
pygame.init()
screen = pygame.display.set_mode([1280,960])
screen.fill([200, 200, 200])

# make A2C agent
agent = A2CAgent(state_size, action_size)

rList, episodes = [], []

# Make Obstacles (obsNumber)
obstaclePos = [[0, 0] for _ in range(obsNumber)]
for i in range(0,obsNumber):
    while True:
        obstaclePos[i][0] = boundaryPos[0] + random.randrange(1, boundaryLength[0])
        obstaclePos[i][1] = boundaryPos[1] + random.randrange(1, boundaryLength[1])
        if obstaclePos[i][0] <= goalPos[0] - agentRadius or obstaclePos[i][0] >= goalPos[0] + agentRadius:
            if obstaclePos[i][1] >= goalPos[1] + agentRadius or obstaclePos[i][1] <= goalPos[1] - agentRadius:
                if obstaclePos[i][0] <= initPosAgent[0] - agentRadius or obstaclePos[i][0] >= initPosAgent[0] + agentRadius:
                    if obstaclePos[i][1] >= initPosAgent[1] + agentRadius or obstaclePos[i][1] <= initPosAgent[1] - agentRadius:
                        break
                
for e in range(num_episodes):
    # Initialize
    done = False
    score = 0
    x = initPosAgent[0]#boundaryPos[0]
    y = initPosAgent[1]#boundaryPos[1]
    print("Episode ", e, "Starts!")
    state = stateGenerator(obstaclePos, [x,y])
    #state = np.reshape(state, [1, state_size])

    while not done:
        
        action = agent.get_action(state)
        
        xMove = 0
        yMove = 0
        
        [xMove, yMove] = takeAction(action)
        
        x = x + xMove
        y = y + yMove

        wallFlag = 0
        collisionFlag = 0
        [x, y, wallFlag] = ckWall(x,y)
        pygame.draw.circle(screen, [100, 100, 255], [x,y], 10, 0)
#         if wallFlag == -1:
#             print("Wall!", action)
        next_state = stateGenerator(obstaclePos, [x,y])

        if(math.sqrt((x - goalPos[0])**2 + (y - goalPos[1])**2) <= 20):
            print("Goal Reached!")           
            collisionFlag = 1
            done = 1
        for i in range(0,obsNumber):
            if moveObstacles:
                obstaclePos[i][0] = obstaclePos[i][0] + random.randrange(-1,2)
                obstaclePos[i][1] = obstaclePos[i][1] + random.randrange(-1,2)
                [obstaclePos[i][0], obstaclePos[i][1], _] = ckWall(obstaclePos[i][0], obstaclePos[i][1])
                obstaclePos[i] = ckInit(initPosAgent ,obstaclePos[i])

            pygame.draw.circle(screen, [255, 50, 50], obstaclePos[i], 10, 0)
            if math.sqrt((x - obstaclePos[i][0])**2 + (y - obstaclePos[i][1])**2) <= 20:
                print("Collision!")
                collisionFlag = -1
                ObjectIndex = i
                done = True 
#         if wallFlag == -1:
#             done = True
            
        if not done:
            reward = -0.1
            if wallFlag == -1:
                reward = -1
        else:
            if collisionFlag == 1:
                reward = 10000
                rList.append(1)
            elif collisionFlag == -1:
                reward = -10000
                rList.append(0)
#             next_state, reward, done, ininitPosAgentfo = env.step(action)
#             next_state = np.reshape(next_state, [1, state_size])
        # if an action make the episode end, then gives penalty of -100
#             reward = reward if not done or score == 499 else -100
        
        agent.train_model(state, action, reward, next_state, done)

        score += reward
        state = next_state

        if done:
            # every episode, plot the play time

            episodes.append(e)
            pylab.plot(episodes, rList, 'b')
            pylab.savefig("./Practice004_DataSave/ActorCriticGraph.png")
        #circle(Surface, color, pos, radius, width=0)
        pygame.draw.circle(screen, [100,255,100], [goalPos[0],goalPos[1]], 10, 2)
        #rect(Surface, color, Rect, width=0)
        pygame.draw.rect(screen, [255,100,100],[boundaryPos[0] - agentRadius, boundaryPos[1] - agentRadius, boundaryLength[0] + agentRadius * 2, boundaryLength[1] + agentRadius * 2],2)
        pygame.display.flip()
        screen.fill([200,200,200])
    print score
    # save the model
    if e % 50 == 0:
        agent.actor.save_weights("./Practice004_DataSave/Actor.h5")
        agent.critic.save_weights("./Practice004_DataSave/Critic.h5")


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               512       
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 1032      
Total params: 1,544
Trainable params: 1,544
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 128)               512       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 129       
Total params: 641
Trainable params: 641
Non-trainable params: 0
_________________________________________________________________
('Episode ', 0, 'Starts!')
Collision!
-10002.3
('Episode ', 1, 'Starts!')
Co

Goal Reached!
9996.5
('Episode ', 143, 'Starts!')
Goal Reached!
9996.5
('Episode ', 144, 'Starts!')
Goal Reached!
9996.5
('Episode ', 145, 'Starts!')
Goal Reached!
9996.5
('Episode ', 146, 'Starts!')
Goal Reached!
9996.3
('Episode ', 147, 'Starts!')
Goal Reached!
9996.5
('Episode ', 148, 'Starts!')
Goal Reached!
9996.3
('Episode ', 149, 'Starts!')
Goal Reached!
9996.5
('Episode ', 150, 'Starts!')
Goal Reached!
9996.5
('Episode ', 151, 'Starts!')
Goal Reached!
9996.5
('Episode ', 152, 'Starts!')
Goal Reached!
9996.5
('Episode ', 153, 'Starts!')
Goal Reached!
9996.5
('Episode ', 154, 'Starts!')
Goal Reached!
9996.5
('Episode ', 155, 'Starts!')
Goal Reached!
9996.5
('Episode ', 156, 'Starts!')
Goal Reached!
9996.5
('Episode ', 157, 'Starts!')
Goal Reached!
9996.5
('Episode ', 158, 'Starts!')
Goal Reached!
9996.5
('Episode ', 159, 'Starts!')
Goal Reached!
9996.5
('Episode ', 160, 'Starts!')
Goal Reached!
9996.5
('Episode ', 161, 'Starts!')
Goal Reached!
9996.5
('Episode ', 162, 'Starts!')


Goal Reached!
9996.5
('Episode ', 307, 'Starts!')
Goal Reached!
9996.5
('Episode ', 308, 'Starts!')
Goal Reached!
9996.5
('Episode ', 309, 'Starts!')
Goal Reached!
9996.5
('Episode ', 310, 'Starts!')
Goal Reached!
9996.5
('Episode ', 311, 'Starts!')
Goal Reached!
9996.5
('Episode ', 312, 'Starts!')
Goal Reached!
9996.5
('Episode ', 313, 'Starts!')
Goal Reached!
9996.5
('Episode ', 314, 'Starts!')
Goal Reached!
9996.5
('Episode ', 315, 'Starts!')
Goal Reached!
9996.5
('Episode ', 316, 'Starts!')
Goal Reached!
9996.5
('Episode ', 317, 'Starts!')
Goal Reached!
9996.5
('Episode ', 318, 'Starts!')
Goal Reached!
9996.5
('Episode ', 319, 'Starts!')
Goal Reached!
9996.5
('Episode ', 320, 'Starts!')
Goal Reached!
9996.5
('Episode ', 321, 'Starts!')
Goal Reached!
9996.5
('Episode ', 322, 'Starts!')
Goal Reached!
9996.5
('Episode ', 323, 'Starts!')
Goal Reached!
9996.5
('Episode ', 324, 'Starts!')
Goal Reached!
9996.5
('Episode ', 325, 'Starts!')
Goal Reached!
9996.5
('Episode ', 326, 'Starts!')


9996.4
('Episode ', 471, 'Starts!')
Goal Reached!
9996.1
('Episode ', 472, 'Starts!')
Goal Reached!
9996.3
('Episode ', 473, 'Starts!')
Goal Reached!
9996.5
('Episode ', 474, 'Starts!')
Goal Reached!
9996.5
('Episode ', 475, 'Starts!')
Goal Reached!
9996.5
('Episode ', 476, 'Starts!')
Goal Reached!
9996.5
('Episode ', 477, 'Starts!')
Goal Reached!
9996.5
('Episode ', 478, 'Starts!')
Goal Reached!
9996.5
('Episode ', 479, 'Starts!')
Goal Reached!
9996.5
('Episode ', 480, 'Starts!')
Goal Reached!
9996.5
('Episode ', 481, 'Starts!')
Goal Reached!
9996.5
('Episode ', 482, 'Starts!')
Goal Reached!
9996.5
('Episode ', 483, 'Starts!')
Goal Reached!
9996.5
('Episode ', 484, 'Starts!')
Goal Reached!
9996.5
('Episode ', 485, 'Starts!')
Goal Reached!
9996.5
('Episode ', 486, 'Starts!')
Goal Reached!
9996.5
('Episode ', 487, 'Starts!')
Goal Reached!
9996.4
('Episode ', 488, 'Starts!')
Goal Reached!
9996.3
('Episode ', 489, 'Starts!')
Goal Reached!
9996.5
('Episode ', 490, 'Starts!')
Goal Reached!


9996.5
('Episode ', 635, 'Starts!')
Goal Reached!
9996.5
('Episode ', 636, 'Starts!')
Goal Reached!
9996.5
('Episode ', 637, 'Starts!')
Goal Reached!
9996.5
('Episode ', 638, 'Starts!')
Goal Reached!
9996.5
('Episode ', 639, 'Starts!')
Goal Reached!
9996.5
('Episode ', 640, 'Starts!')
Goal Reached!
9996.5
('Episode ', 641, 'Starts!')
Goal Reached!
9996.5
('Episode ', 642, 'Starts!')
Goal Reached!
9996.5
('Episode ', 643, 'Starts!')
Goal Reached!
9996.5
('Episode ', 644, 'Starts!')
Goal Reached!
9996.5
('Episode ', 645, 'Starts!')
Goal Reached!
9996.5
('Episode ', 646, 'Starts!')
Goal Reached!
9996.5
('Episode ', 647, 'Starts!')
Goal Reached!
9996.5
('Episode ', 648, 'Starts!')
Goal Reached!
9996.5
('Episode ', 649, 'Starts!')
Goal Reached!
9996.5
('Episode ', 650, 'Starts!')
Goal Reached!
9996.5
('Episode ', 651, 'Starts!')
Goal Reached!
9996.5
('Episode ', 652, 'Starts!')
Goal Reached!
9996.5
('Episode ', 653, 'Starts!')
Goal Reached!
9996.5
('Episode ', 654, 'Starts!')
Goal Reached!


...

Goal Reached!
9996.5
('Episode ', 2253, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2254, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2255, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2256, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2257, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2258, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2259, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2260, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2261, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2262, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2263, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2264, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2265, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2266, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2267, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2268, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2269, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2270, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2271, 'Starts!')
Goal Reached!
9996.3
('Episode 

9996.5
('Episode ', 2414, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2415, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2416, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2417, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2418, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2419, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2420, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2421, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2422, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2423, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2424, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2425, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2426, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2427, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2428, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2429, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2430, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2431, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2432, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2433, 'Star

Goal Reached!
9996.5
('Episode ', 2576, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2577, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2578, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2579, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2580, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2581, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2582, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2583, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2584, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2585, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2586, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2587, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2588, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2589, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2590, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2591, 'Starts!')
Goal Reached!
9996.4
('Episode ', 2592, 'Starts!')
Goal Reached!
9996.3
('Episode ', 2593, 'Starts!')
Goal Reached!
9995.7
('Episode ', 2594, 'Starts!')
Collision!
-10002.9
('Episode '

Goal Reached!
9996.5
('Episode ', 2737, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2738, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2739, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2740, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2741, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2742, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2743, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2744, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2745, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2746, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2747, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2748, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2749, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2750, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2751, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2752, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2753, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2754, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2755, 'Starts!')
Goal Reached!
9996.5
('Episode 

Goal Reached!
9996.5
('Episode ', 2898, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2899, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2900, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2901, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2902, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2903, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2904, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2905, 'Starts!')
Goal Reached!
9996.4
('Episode ', 2906, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2907, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2908, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2909, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2910, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2911, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2912, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2913, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2914, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2915, 'Starts!')
Goal Reached!
9996.5
('Episode ', 2916, 'Starts!')
Goal Reached!
9996.5
('Episode 

In [ ]:
print("Percent of successful episodes: " + str(100.0 * sum(rList)/num_episodes) + "%")

plt.bar(range(len(rList)), rList, color = "Blue", width = 0.00001)
plt.show()

Percent of successful episodes: 23.322%
